In [1]:
from test2 import Model
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import pandas as pd
from Preprocessing import Data
from Preprocessing import Preprocessing
from Classifier import Classifier
from Dummy import Analysis

#%matplotlib inline

In [2]:
class Prepare(object):
    def __init__(self, source='../input/raw_data.txt'):
        self.preprocess_data(source)
        self.perform_classfier()
        self.perform_analysis()
    
    def preprocess_data(self, source):
        preprocessing = Preprocessing(source)
        preprocessing.Incomplete_wafer(preprocessing.data)
        preprocessing.wafer_count(preprocessing.data, 23)
        print(len(preprocessing.data))
        preprocessing.chamber_variety(preprocessing.data, 20)
        print(len(preprocessing.data))
        preprocessing.period(preprocessing.data)
        print(len(preprocessing.data))
        preprocessing.main(preprocessing.data)
        print(len(preprocessing.data))
        self.preprocessed_data = preprocessing.data
        print(len(self.preprocessed_data))
    
    def perform_classfier(self):
        classifier = Classifier(self.preprocessed_data)
        classifier.create_wafer_data()
        print("created wafer_data.csv")
        print(len(classifier.wafer_data))
        classifier.wafer_data.to_csv('../result/wafer_data.csv', sep=',', index = False)
        classifier.relative_ratio_mean_criterion(80,20)
        
        classifier.create_wafer_BOB_WOW_group()
        print("created wafer_BOB_WOW_Group.csv")
        print(len(classifier.wafer_BOB_WOW_Group))
        classifier.wafer_BOB_WOW_Group.to_csv('../result/wafer_BOB_WOW_group.csv', sep=',')
        classifier.create_chamber_perf()
        print("created chamber_perf")
        print(len(classifier.chamber_perf))
        classifier.chamber_perf.to_csv('../result/chamber_perf.csv', sep=',')
        
    def perform_analysis(self):
        print("Analysis begins")
        analyzer = Analysis()
        print("KS_test begins")
        analyzer.create_KS_test()
        print("KS_test completed")
        print("ANOVA begins")
        analyzer.create_ANOVA()
        print("ANOVA completed")
        print("KRUSKAL begins")
        analyzer.create_Kruskal()
        print("KRUSKAL completed")
        print("Linear_Contrasts begins")
        analyzer.create_stat_analysis()
        print("Linear_Contrasts completed")
        print("Multiple t-test begins")
        analyzer.create_mt_stat_test()
        print("Multiple t-test completed")
        print("User_defined test begins")
        analyzer.create_user_defined_analysis()
        print("User_defined test completed")
        analyzer.create_combined_results()
        print("create chamber_info")
        analyzer.create_chamber_info()
        #print("create chamber_comparison")
        #analyzer.create_chamber_comparison()

In [30]:
class STEP_GRAPH(object):
    def __init__(self):
        self.model = Model() 
        self.high_subset = self.model.high_subset
        self.low_subset = self.model.low_subset
        self.machine_list = self.high_subset + self.low_subset
        self.wafer_data = pd.read_csv('../result/wafer_data.csv', index_col = False)
    def assign_step(self):
        self.machine_dict = {}
        for machine in self.machine_list:
            step = machine[:8]
            if step in self.machine_dict.keys():
                self.machine_dict[step].append(machine)
            else:
                self.machine_dict[step] = [machine]
                
    def calculate_range(self):
        pass
    
    def create_line_graph(self):
        def change_index_value(x):
            new_index = []
            for index in x.tolist():
                new_index.append(index[5:])
            return new_index
        
        for step in self.machine_dict.keys():
            machines = self.machine_dict[step]
            #step_machine_value = self.wafer_data.loc[self.wafer_data['S_E_CHAMBER_ID'] == machines[0],['TKIN_TIME','VALUE']]
            #step_machine_value = step_machine_value.groupby('TKIN_TIME').mean()
            count = 0
            min_value = 9999
            for machine in machines:
                if count == 0:
                    step_machine_value = self.wafer_data.loc[self.wafer_data['S_E_CHAMBER_ID'] == machine,['TKIN_TIME','VALUE']].groupby('TKIN_TIME').mean()
                    
                    if min_value > min(step_machine_value['VALUE']):
                        min_value = min(step_machine_value['VALUE'])
                    
                    if machine in self.high_subset:
                        step_machine_value.columns = ['BOB_' + machine]
                    else:
                        step_machine_value.columns = ['WOW_' + machine]
                else:
                    temp = self.wafer_data.loc[self.wafer_data['S_E_CHAMBER_ID'] == machine,['TKIN_TIME','VALUE']]
                    temp = temp.groupby('TKIN_TIME').mean()
                    
                    if min_value > min(temp['VALUE']):
                        min_value = min(temp['VALUE'])
                    
                    if machine in self.high_subset:
                        temp.columns = ['BOB_' + machine]
                    else:
                        temp.columns = ['WOW_' + machine]
                    step_machine_value = pd.concat([step_machine_value,temp], axis=1)
                count+=1
            step_machine_value.index = change_index_value(step_machine_value.index)
            step_machine_value = step_machine_value.fillna(method='ffill')
            fig = plt.figure(); 
            font = {'family': 'normal', 'weight': 'bold', 'size': 3}
            plt.rc('font', **font)
            ax = step_machine_value.plot(style='-');
            #plt.legend(loc='best', prop={'size':6})
            plt.ylim(ymin=min_value-1)
            
            box = ax.get_position()
            ax.set_position([box.x0, box.y0, box.width *0.8, box.height])
            ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
            
            plt.savefig('../result/graph/{}.png'.format(step), dpi=300)
            
    def create_scatter_graph(self):
        def change_index_value(x):
            new_index = []
            for index in x.tolist():
                new_index.append(index[5:])
            return new_index
        
        for step in self.machine_dict.keys():
            machines = self.machine_dict[step]
            #step_machine_value = self.wafer_data.loc[self.wafer_data['S_E_CHAMBER_ID'] == machines[0],['TKIN_TIME','VALUE']]
            #step_machine_value = step_machine_value.groupby('TKIN_TIME').mean()
            count = 0
            min_value = 9999
            for machine in machines:
                if count == 0:
                    step_machine_value = self.wafer_data.loc[self.wafer_data['S_E_CHAMBER_ID'] == machine,['TKIN_TIME','VALUE']].groupby('TKIN_TIME').mean()
                    
                    if min_value > min(step_machine_value['VALUE']):
                        min_value = min(step_machine_value['VALUE'])
                        
                    if machine in self.high_subset:
                        step_machine_value.columns = ['BOB_' + machine]
                    else:
                        step_machine_value.columns = ['WOW_' + machine]
                        
                    
                else:
                    temp = self.wafer_data.loc[self.wafer_data['S_E_CHAMBER_ID'] == machine,['TKIN_TIME','VALUE']]
                    temp = temp.groupby('TKIN_TIME').mean()
                    
                    if min_value > min(temp['VALUE']):
                        min_value = min(temp['VALUE'])
                        
                    if machine in self.high_subset:
                        temp.columns = ['BOB_' + machine]
                    else:
                        temp.columns = ['WOW_' + machine]
                    step_machine_value = pd.concat([step_machine_value,temp], axis=1)
                count+=1
            step_machine_value.index = change_index_value(step_machine_value.index)
            #step_machine_value = step_machine_value.fillna(method='ffill')
            fig = plt.figure();
            font = {'family': 'normal', 'weight': 'bold', 'size': 3}
            plt.rc('font', **font)
            ax = step_machine_value.plot(style='.-');
            #plt.legend(loc='best', prop={'size':6})
            #print(min_value)
            plt.ylim(ymin=min_value-1)
            
            box = ax.get_position()
            ax.set_position([box.x0, box.y0, box.width *0.8, box.height])
            ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
            
            plt.savefig('../result/graph/{}.png'.format(step), dpi=300)

In [4]:
prepare = Prepare()

DATA class
1121347
processing begins
process 1 finished
1098877


C:\Users\ProDesk\Desktop\LAB\SAMSUNG_PROJECT\IMPLE\codebase\Preprocessing.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  Preprocessed_data_2['CHAMBER_ID'][idx] = value


process 2 finished
1098877
process 3 finished
1095269
PREPROCESSING class
1095269
942379
942379
323804
115192
115192
CLASSIFIER class


C:\Users\ProDesk\Desktop\LAB\SAMSUNG_PROJECT\IMPLE\codebase\Classifier.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.wafer_data['TKIN_TIME'] =  self.wafer_data['TKIN_TIME'].map(lambda t: t.strftime('%Y-%m-%d %H'))


created wafer_data.csv
114847
created wafer_BOB_WOW_Group.csv
114847
created chamber_perf
501
Analysis begins
KS_test begins
KS_test completed
ANOVA begins
ANOVA completed
KRUSKAL begins
KRUSKAL completed
Linear_Contrasts begins


C:\Users\ProDesk\Anaconda3\envs\python2\lib\site-packages\scipy\stats\stats.py:4660: DeprecationWarning: Calling `mannwhitneyu` without specifying `alternative` is deprecated.
  "`alternative` is deprecated.", DeprecationWarning)
C:\Users\ProDesk\Desktop\LAB\SAMSUNG_PROJECT\IMPLE\codebase\Dummy.py:212: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.chamber_stat_LC['LC_p_value'][row.name]  = p_value
C:\Users\ProDesk\Desktop\LAB\SAMSUNG_PROJECT\IMPLE\codebase\Dummy.py:215: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.chamber_stat_LC['LCresult'][row.name] = 'BOB'
C:\Users\ProDesk\Desktop\LAB\SAMSUNG_PROJECT\IMPLE\codebase\Dummy.py:217: SettingWi

Linear_Contrasts completed
Multiple t-test begins


C:\Users\ProDesk\Desktop\LAB\SAMSUNG_PROJECT\IMPLE\codebase\Dummy.py:344: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.chamber_stat_MTTest['t_test_result'][BOB] = 'BOB'
C:\Users\ProDesk\Desktop\LAB\SAMSUNG_PROJECT\IMPLE\codebase\Dummy.py:345: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.chamber_stat_MTTest['t_test_result'][WOW] = 'WOW'


Multiple t-test completed
User_defined test begins
User_defined test completed
create chamber_info


C:\Users\ProDesk\Anaconda3\envs\python2\lib\site-packages\pandas\tools\merge.py:536: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


## MAIN

In [63]:


graph = STEP_GRAPH()
graph.assign_step()
#Line graph를 원하는 경우
graph.create_line_graph()
#Scatter-line graph를 원하는 경우
#graph.create_scatter_graph()

C:\Users\ProDesk\Desktop\LAB\SAMSUNG_PROJECT\IMPLE\codebase\test2.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.wafer_info_dummy['FROM'][self.wafer_info_dummy['FROM'].isin(self.wafer_group.loc[self.wafer_group['LCresult'] == 'DUMMY','S_E_CHAMBER_ID'])] = self.wafer_info_dummy['FROM'][self.wafer_info_dummy['FROM'].isin(self.wafer_group.loc[self.wafer_group['LCresult'] == 'DUMMY','S_E_CHAMBER_ID'])].apply(dummy)
C:\Users\ProDesk\Desktop\LAB\SAMSUNG_PROJECT\IMPLE\codebase\test2.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.wafer_info_dummy['TO'][self.wafer_info_dummy['TO'].isin(self.wafer_group.loc[self.wafer_group['LCresult'] == '